# Convolution Neural Network

In [1]:
import deeplake
import numpy as np

import tqdm
import random
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from collections import OrderedDict

/usr/local/lib/python3.11/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.4) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
print(torch.__version__)

2.1.0


In [16]:
from dataprocessing import DataLoaderWrapper

data=DataLoaderWrapper(batch_size=32)
celebA_train,celebA_val,celebA_test=data.initialize_celebA_dataloaders()
adience=data.initialize_adience_dataloaders()

-

Opening dataset in read-only mode as you don't have write permissions.


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/adience



/

...

Opening dataset in read-only mode as you don't have write permissions.


\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/celeb-a-test



\

hub://activeloop/celeb-a-test loaded successfully.



In [4]:
class Levi_Hassner(nn.Module):
    def __init__(self,output=2,deformable=False) -> None:
        super().__init__()
        self.deformable=deformable

        self.layers=nn.Sequential(OrderedDict([
            # first convolutional layer
            ('conv1',nn.Conv2d(3, 96, 7, padding='valid', stride=4)),  # No padding
            ('relu1',nn.ReLU()),
            ('maxpool1',nn.MaxPool2d(3, stride=2)),  # Max pooling over a (3, 3) window with 2 pixel stride)
            ('lrn1',nn.LocalResponseNorm(size=5, k=2, alpha=10**(-4), beta=0.75)),

            # second convolutional layer
            ('conv2',nn.Conv2d(96, 256, 5, padding='same')), # Same padding
            ('relu2',nn.ReLU()),
            ('maxpool2',nn.MaxPool2d(3, stride=2)),  # Max pooling over a (3, 3) window with 2 pixel stride)
            ('lrn2',nn.LocalResponseNorm(size=5, k=2, alpha=10**(-4), beta=0.75)),

            # third convolutional layer
            ('conv3',nn.Conv2d(256, 384, 3, padding='same')),  # Same padding
            ('relu3',nn.ReLU()),
            ('maxpool3',nn.MaxPool2d(3, stride=2)),  # Max pooling over a (3, 3) window with 2 pixel stride)
            ('flatten',nn.Flatten()),

            ('fc1',nn.Linear(384*6*6, 512)), # input 384 * 6 * 6 = 13824, output 512
            ('relu4',nn.ReLU()),
            ('dropout1',nn.Dropout(0.5)),

            ('fc2',nn.Linear(512,512)),
            ('relu5',nn.ReLU()),
            ('dropout2',nn.Dropout(0.5)),
            
            ('fc3',nn.Linear(512,output)), # output = number of classes 
        ]))
        self.prob=nn.Softmax(dim=1) # new stuff to check if its causes harm

    def forward(self,x):
        x=self.layers(x)
        prob=self.prob(x)
        return prob

In [5]:
gender_model = Levi_Hassner()
gender_model

Levi_Hassner(
  (layers): Sequential(
    (conv1): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=valid)
    (relu1): ReLU()
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (lrn1): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (relu2): ReLU()
    (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (lrn2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (relu3): ReLU()
    (maxpool3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (fc1): Linear(in_features=13824, out_features=512, bias=True)
    (relu4): ReLU()
    (dropout1): Dropout(p=0.5, inplace=False)
    (fc2): Linear(in_features=512, out_features=512, bias=True)
    (relu5): ReLU()
    (

In [6]:
age_model = Levi_Hassner(output=8)
age_model

Levi_Hassner(
  (layers): Sequential(
    (conv1): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=valid)
    (relu1): ReLU()
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (lrn1): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (relu2): ReLU()
    (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (lrn2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (relu3): ReLU()
    (maxpool3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (fc1): Linear(in_features=13824, out_features=512, bias=True)
    (relu4): ReLU()
    (dropout1): Dropout(p=0.5, inplace=False)
    (fc2): Linear(in_features=512, out_features=512, bias=True)
    (relu5): ReLU()
    (

In [17]:
for i,data in enumerate(celebA_train):
    print(data['images'].shape)
    print(data['male'].shape)
    print(data['young'].shape)
    

/usr/local/lib/python3.11/site-packages/deeplake/integrations/pytorch/common.py:126: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


torch.Size([32, 3, 227, 227])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 3, 227, 227])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 3, 227, 227])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 3, 227, 227])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 3, 227, 227])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 3, 227, 227])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 3, 227, 227])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 3, 227, 227])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 3, 227, 227])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 3, 227, 227])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 3, 227, 227])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 3, 227, 227])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 3, 227, 227])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 3, 227, 227])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 3, 2

KeyboardInterrupt: 

In [25]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 53 * 53, 120)  # Adjust the input dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net=Net()

In [21]:
model=Levi_Hassner()

In [32]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gender_model.parameters(), lr=1e-3)

for epoch in range(2):
    running_loss=0.0
    for i,data in enumerate(celebA_train):
        inputs, labels = data['images'],data['male']
        labels=torch.argmax(labels,dim=1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        print(running_loss)
        if i==2:break
        # if i % 2000 == 1999:    # print every 2000 mini-batches
        #     print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
        #     running_loss = 0.0

/usr/local/lib/python3.11/site-packages/deeplake/integrations/pytorch/common.py:126: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


0.6783455014228821
1.3590947985649109
2.036963641643524
0.681087851524353
1.3592827320098877
2.036513566970825


In [7]:
# Choose a loss function for training
# gender_model = Levi_Hassner(output=1)
loss_object = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gender_model.parameters(), lr=1e-3)
EPOCHS = 30

# Train step
def train_step(images, labels, optimizer, model):
    optimizer.zero_grad()
    predictions = model(images)
    predictions = predictions[:, 0].unsqueeze(1)
    
    loss = loss_object(predictions.float(), labels.float())
    loss.backward()
    optimizer.step()
    predictions = (predictions >= 0.5).float()
    return loss.item(), (predictions == labels).type(torch.float).mean().item()

# def test_step(images, labels, model):
#     predictions = model(images)
#     predictions = predictions[:, 0].unsqueeze(1)
#     t_loss = loss_object(predictions.float(), labels.float())
#     predictions = (predictions >= 0.5).float()
#     return t_loss.item(), (predictions == labels).type(torch.float).mean().item()

for epoch in range(EPOCHS):
        train_loss = 0.0
        train_accuracy = 0.0
        test_loss = 0.0
        test_accuracy = 0.0
        train_steps = 0
        test_steps = 0

        for images, age_labels, gender_labels in celebA_train:
            l, a = train_step(images, gender_labels, optimizer, gender_model)
            train_loss += l
            train_accuracy += a
            train_steps += 1

        # with torch.no_grad():
        #     for test_images, age_labels, gender_labels in celebA_val:
        #         t_l, t_a = test_step(test_images, gender_labels, gender_model)
        #         test_loss += t_l
        #         test_accuracy += t_a
        #         test_steps += 1
        #         test_acc.append(test_accuracy / test_steps)
        # template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
        # print(template.format(epoch + 1,
        #                       train_loss / train_steps,
        #                       train_accuracy / train_steps,
        #                       test_loss / test_steps,
        #                       test_accuracy / test_steps))


/usr/local/lib/python3.11/site-packages/deeplake/integrations/pytorch/common.py:126: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


KeyboardInterrupt: 

# Transfer Learning

In [ ]:
for param in gender_model.parameters():
    param.requires_grad = False
for param in gender_model.fc3.parameters(): # unfreeze weights of last layer
    param.requires_grad = True

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gender_model.fc3.parameters(), lr=0.001)

# Training loop
for epoch in range(5):
    optimizer.zero_grad()
    outputs = resnet18(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}/5, Loss: {loss.item()}')